In [45]:
import httpx
import time
import pandas as pd

In [ ]:
## Functions I might need later:

# def dedupe(xs):
#     return list(dict.fromkeys([x for x in xs if x]))

# env_post_ids = dedupe(env_post_ids)
# infr_post_ids = dedupe(infr_post_ids)
# housing_post_ids = dedupe(housing_post_ids)
# econ_post_ids = dedupe(econ_post_ids)
# life_qual_post_ids = dedupe(life_qual_post_ids)
# aesth_post_ids = dedupe(aesth_post_ids)
# gov_post_ids = dedupe(gov_post_ids)
# not_useful_post_ids = dedupe(not_useful_post_ids)


## REMOVE URLS IN POSTS
## 

In [59]:
MAX_POSTS = 500

all_post_ids = []

HEADERS = {
    "User-Agent": "research-script/0.1 (academic, non-commercial)"
}

subreddit = "nova"
search_url = f"https://www.reddit.com/r/{subreddit}/search.json"

params = {
    "q": 'datacenter',
    "restrict_sr": 1,
}

env_post_ids = []
infr_post_ids = []
housing_post_ids = []
econ_post_ids = []
life_qual_post_ids = []
aesth_post_ids = []
gov_post_ids = []
not_useful_post_ids = []

after = None

environmental_keywords = ["pollution", "emissions", "diesel", "fumes", "light pollution", "water", "drought", "water consumption",  "waste", "carbon", "climate"]
infrastructure_utilities_keywords = ["utilities", "electric", "power", "grid", "blackout", "substation", "reliable", "reliability", "capacity", "infrastructure", "generator", "water"]
housing_keywords = ["housing", "house", "home", "rent", "property", "real estate", "house price", "housing price", "housing cost", "land cost", "land price", "gentrification"]
economic_keywords = ["job", "work", "revenue", "tax", "money", "rich", "salary", "economy", "investment", "business"]
life_quality_keywords = ["resident", "fumes", "noise", "loud", "vibration", "light pollution", "sound", "smell", "traffic"]
aesthetics_keywords = ["landscape", "visual impact",  "aesthetics", "looks", "architecture", "design", "facade", "industrial", "windows", "dark"]
government_keywords = ["zoning", "planning", "approval", "permit", "city council"]

# NEED TO INCLUDE THIS FILTERING
# dont_include = ["server", "rack", "switch", "firewall", "BGP", "latency"]

with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
    while True:
        if after is not None:
            params["after"] = after
        else:
            params.pop("after", None)

        r = client.get(search_url, params=params)
        print("REQUEST URL:", str(r.url))
        r.raise_for_status()
        listing = r.json()

        children = listing["data"]["children"]
        if not children:
            break

        for child in children:
            # data = child.get("data", {})
            # post_id = data.get("id")

            # title = (data.get("title") or "")
            # selftext = (data.get("selftext") or "")
            # text = (title + "\n" + selftext).lower()

            post_id = child["data"]["id"]
            all_post_ids.append(post_id)
            title = (child["data"]["title"] or "")
            selftext = (child["data"]["selftext"] or "")
            text = (title + " " + selftext).lower()

            matched = False
        
            if any(kw in text for kw in environmental_keywords):
                env_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in infrastructure_utilities_keywords):
                infr_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in housing_keywords):
                housing_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in economic_keywords):
                econ_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in life_quality_keywords):
                life_qual_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in aesthetics_keywords):
                aesth_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in government_keywords):
                gov_post_ids.append(post_id)
                matched = True
            if not matched:
                not_useful_post_ids.append(post_id)
            
        after = listing["data"]["after"]
        print("After:", after)
        if after is None:
            break

        if len(all_post_ids) >= MAX_POSTS:
            break

        time.sleep(0.2)

print("Total posts scanned:", len(all_post_ids))
print("Found environmental posts:", len(env_post_ids))
print("Found infrastructure posts:", len(infr_post_ids))
print("Found housing posts:", len(housing_post_ids))
print("Found economic posts:", len(econ_post_ids))
print("Found life quality posts:", len(life_qual_post_ids))
print("Found aesthetic posts:", len(aesth_post_ids))
print("Found government posts:", len(gov_post_ids))

REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1
After: t3_esy3zu
REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1&after=t3_esy3zu
After: t3_j25zf4
REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1&after=t3_j25zf4
After: None
Total posts scanned: 51
Found environmental posts: 2
Found infrastructure posts: 8
Found housing posts: 12
Found economic posts: 14
Found life quality posts: 11
Found aesthetic posts: 2
Found government posts: 3


In [62]:
with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
        pid = "1qborpq"
        post_url = f"https://www.reddit.com/comments/{pid}.json"                
        r = client.get(post_url)            
        post_json = r.json()
        text = post_json[0]["data"]["children"][0]["data"]["selftext"] + post_json[0]["data"]["children"][0]["data"]["title"]
if any(kw in text for kw in environmental_keywords):
    print("Environmental")
    matched = True
if any(kw in text for kw in infrastructure_utilities_keywords):
    print("Infrastructure")
    matched = True
if any(kw in text for kw in housing_keywords):
    print("Housing")
    matched = True
if any(kw in text for kw in economic_keywords):
    print("Economic")
    matched = True
if any(kw in text for kw in life_quality_keywords):
    print("Life quality")
    matched = True
if any(kw in text for kw in aesthetics_keywords):
    print("Aesthetics")
    matched = True
if any(kw in text for kw in government_keywords):
    print("Governmental")
    matched = True
if matched == False:
    print("None")

import re
from transformers import pipeline
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True,      # <--- important
    padding=True,         # <--- for batching
    max_length=512   
)

results = classifier(text, truncation=True, padding=True, max_length=512, batch_size=32)
print(results)

Infrastructure


Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.7456856369972229}]


In [48]:
#listing -> data -> children (only 1 when going to a specific post) -> data -> selftext/created/id
#comments:children - data - body

#comments have to be above 200 characters? - some that are under express good opinions so idk (maybe over 75?)

In [ ]:
nova_theme_lists = [env_post_ids, infr_post_ids, housing_post_ids, econ_post_ids, life_qual_post_ids, aesth_post_ids, gov_post_ids]
posts = pd.DataFrame(columns=["ids", "text", "date", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])

for theme in nova_theme_lists:
    df1 = pd.DataFrame(columns=["ids", "text", "date", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])
    post_ids = []
    post_texts = []
    post_dates = []
    with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
        for pid in theme:
            post_url = f"https://www.reddit.com/comments/{pid}.json"                
            r = client.get(post_url)
            r.raise_for_status()
            post_json = r.json()
            post_ids.append(pid)
            text_and_title = post_json[0]["data"]["children"][0]["data"]["selftext"] + post_json[0]["data"]["children"][0]["data"]["title"]
            post_texts.append(text_and_title)
            post_dates.append(post_json[0]["data"]["children"][0]["data"]["created_utc"])
    df1["ids"] = post_ids
    df1["text"] = post_texts
    df1["date"] = post_dates
    if theme == env_post_ids:
        df1["environment"] = True
    if theme == infr_post_ids:
        df1["infrastructure"] = True
    if theme == housing_post_ids:
        df1["housing"] = True
    if theme == econ_post_ids:
        df1["economy"] = True
    if theme == life_qual_post_ids:
        df1["life quality"] = True
    if theme == aesth_post_ids:
        df1["aesthetics"] = True
    if theme == gov_post_ids:
        df1["government"] = True
    posts = pd.concat([posts, df1], ignore_index=True)


print(posts)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_44576/1697576501.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  posts = pd.concat([posts, df1], ignore_index=True)


        ids                                               text          date  \
0    7htslh  Odd one of the day.\n\nJust me fishing. I have...  1.512516e+09   
1   14gh0c1  The applicant also wants a special exception t...  1.687473e+09   
2    xae8nw  Virginia has more datacenter capacity than Eur...  1.662779e+09   
3    wktuak  It seems there's a solid possibility I will ge...  1.660127e+09   
4    eddta7  Everybody in this area knows that datacenters ...  1.576866e+09   
5    7htslh  Odd one of the day.\n\nJust me fishing. I have...  1.512516e+09   
6   1po4dfm  Hello! I am a reporter based in DC looking for...  1.765898e+09   
7   1gawpld  Is it time to to slow down Datacenter growth a...  1.729754e+09   
8   14gh0c1  The applicant also wants a special exception t...  1.687473e+09   
9   15cwyr6  I feel like I hear lots of whining from Loudon...  1.690649e+09   
10   7htslh  Odd one of the day.\n\nJust me fishing. I have...  1.512516e+09   
11  1isnvbc  Real Estate Tax Hike, 208 S

In [50]:
grouping_cols = ["ids", "text", "date"]
posts = posts.groupby(grouping_cols, as_index=False).max()
theme_cols = ["environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"]
posts[theme_cols] = posts[theme_cols].fillna(False)
posts.head(15)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_44576/3906843801.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  posts[theme_cols] = posts[theme_cols].fillna(False)


,ids,text,date,environment,infrastructure,housing,economy,life quality,aesthetics,government
0,10s4xtb,Where did you move to or planning to move to ?...,1.675382e+09,False,False,False,False,False,False,True
1,117zom7,I posted this somewhere else but I am curious ...,1.676973e+09,False,False,True,True,False,False,False
2,1341j3h,"Hi, I'm a local youtuber planning on making a ...",1.682885e+09,False,False,False,False,True,False,True
3,14gh0c1,The applicant also wants a special exception t...,1.687473e+09,True,True,True,False,False,False,False
4,152m2sz,"Citing Loudoun County property records, [BizJo...",1.689648e+09,False,False,True,False,True,False,False
5,158wywd,\n\nOff loco pkwy before the greenway. Hope no...,1.690257e+09,False,False,True,False,True,False,False
6,15cwyr6,I feel like I hear lots of whining from Loudon...,1.690649e+09,False,False,True,True,True,False,False
7,171nn12,"Hello,\n\nI currently work for a cox contracto...",1.696626e+09,False,False,True,True,False,False,False
8,1al34nd,"I am seeking talking points, and their sources...",1.707313e+09,False,False,True,False,False,False,False
9,1bdp55a,The board’s land use policy committee heard a ...,1.710329e+09,False,False,True,False,True,True,True


In [51]:
import re
from transformers import pipeline
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True,      # <--- important
    padding=True,         # <--- for batching
    max_length=512        
)

Device set to use mps:0


In [52]:
texts = posts["text"].astype(str).tolist()
results = classifier(texts, truncation=True, padding=True, max_length=512, batch_size=32)
posts["roberta"] = results

In [53]:
posts["name"] = posts["roberta"].apply(lambda x: x["label"])

def roberta_label(name):
    if name == 'LABEL_2':
        return "positive"
    elif name == 'LABEL_1':
        return "neutral"
    else:
        return "negative"
posts["label"] = posts["name"].apply(roberta_label)

posts["degree"] = posts["roberta"].apply(lambda x: x["score"])
posts = posts.drop(["roberta", "name"], axis=1)

In [54]:
posts.head(15)

,ids,text,date,environment,infrastructure,housing,economy,life quality,aesthetics,government,label,degree
0,10s4xtb,Where did you move to or planning to move to ?...,1.675382e+09,False,False,False,False,False,False,True,neutral,0.735660
1,117zom7,I posted this somewhere else but I am curious ...,1.676973e+09,False,False,True,True,False,False,False,negative,0.818258
2,1341j3h,"Hi, I'm a local youtuber planning on making a ...",1.682885e+09,False,False,False,False,True,False,True,neutral,0.810689
3,14gh0c1,The applicant also wants a special exception t...,1.687473e+09,True,True,True,False,False,False,False,neutral,0.657676
4,152m2sz,"Citing Loudoun County property records, [BizJo...",1.689648e+09,False,False,True,False,True,False,False,neutral,0.836115
5,158wywd,\n\nOff loco pkwy before the greenway. Hope no...,1.690257e+09,False,False,True,False,True,False,False,neutral,0.534903
6,15cwyr6,I feel like I hear lots of whining from Loudon...,1.690649e+09,False,False,True,True,True,False,False,negative,0.562770
7,171nn12,"Hello,\n\nI currently work for a cox contracto...",1.696626e+09,False,False,True,True,False,False,False,negative,0.783623
8,1al34nd,"I am seeking talking points, and their sources...",1.707313e+09,False,False,True,False,False,False,False,negative,0.626999
9,1bdp55a,The board’s land use policy committee heard a ...,1.710329e+09,False,False,True,False,True,True,True,neutral,0.826911


In [ ]:
# calculating average sentiment based on theme:

# 1. Weigh all positive, negative, and neutral posts equally regardless of the degree of confidence
def avg_sentiment_calculation_1(theme):
    theme_posts = posts[posts[theme] == True]
    sum_of_sentiments = len(theme_posts.loc[theme_posts["label"] == "positive"]) - len(theme_posts.loc[theme_posts["label"] == "negative"])
    number_of_posts = len(theme_posts)
    return sum_of_sentiments/number_of_posts


# 2. Multiply sentiment of post (-1/0/1) by their degree of confidence, means that positive and negative posts will have a diminished effect while neutral posts will remain the same
def avg_sentiment_calculation_2(theme):
    theme_posts = posts[posts[theme] == True]
    sum_of_degrees = theme_posts.loc[theme_posts["label"] == "positive", "degree"].sum() - theme_posts.loc[theme_posts["label"] == "negative", "degree"].sum()
    number_of_posts = len(theme_posts)
    return sum_of_degrees/number_of_posts


# 3. Weigh the positive and negative posts by degree of confidence in the numerator, and then weigh neutral posts by degree of confidence in the demonimator (so the 0s matter less if the degree of confidence is less)
def avg_sentiment_calculation_3(theme):
    theme_posts = posts[posts[theme] == True]
    sum_of_degrees = theme_posts.loc[theme_posts["label"] == "positive", "degree"].sum() - theme_posts.loc[theme_posts["label"] == "negative", "degree"].sum()
    weighed_denominator = len(theme_posts.loc[theme_posts["label"] == "positive"]) + len(theme_posts.loc[theme_posts["label"] == "negative"]) + theme_posts.loc[theme_posts["label"] == "neutral", "degree"].sum()
    return sum_of_degrees/weighed_denominator


# 4. Weigh all posts by their degree in the numerator and denominator, means that the average sentiment will just be +/- 1 if there are only positive or negative themes but other than that does a pretty good job of weighing neutrality
def avg_sentiment_calculation_4(theme):
    theme_posts = posts[posts[theme] == True]
    pos = theme_posts.loc[theme_posts["label"] == "positive", "degree"].sum()
    neg = theme_posts.loc[theme_posts["label"] == "negative", "degree"].sum()
    total = theme_posts["degree"].sum()
    return (pos-neg)/total

print("Average calculation method 1:")
print("Average sentiment of environmental posts:", avg_sentiment_calculation_1("environment"))
print("Average sentiment of infrastructural posts:", avg_sentiment_calculation_1("infrastructure"))
print("Average sentiment of housing-related posts:", avg_sentiment_calculation_1("housing"))
print("Average sentiment of economic posts:", avg_sentiment_calculation_1("economy"))
print("Average sentiment of life-quality-related posts:", avg_sentiment_calculation_1("life quality"))
print("Average sentiment of aesthetics-related posts:", avg_sentiment_calculation_1("aesthetics"))
print("Average sentiment of governmental posts:", avg_sentiment_calculation_1("government"))
print("\n")

print("Average calculation method 2:")
print("Average sentiment of environmental posts:", avg_sentiment_calculation_2("environment"))
print("Average sentiment of infrastructural posts:", avg_sentiment_calculation_2("infrastructure"))
print("Average sentiment of housing-related posts:", avg_sentiment_calculation_2("housing"))
print("Average sentiment of economic posts:", avg_sentiment_calculation_2("economy"))
print("Average sentiment of life-quality-related posts:", avg_sentiment_calculation_2("life quality"))
print("Average sentiment of aesthetics-related posts:", avg_sentiment_calculation_2("aesthetics"))
print("Average sentiment of governmental posts:", avg_sentiment_calculation_2("government"))
print("\n")

print("Average calculation method 3:")
print("Average sentiment of environmental posts:", avg_sentiment_calculation_3("environment"))
print("Average sentiment of infrastructural posts:", avg_sentiment_calculation_3("infrastructure"))
print("Average sentiment of housing-related posts:", avg_sentiment_calculation_3("housing"))
print("Average sentiment of economic posts:", avg_sentiment_calculation_3("economy"))
print("Average sentiment of life-quality-related posts:", avg_sentiment_calculation_3("life quality"))
print("Average sentiment of aesthetics-related posts:", avg_sentiment_calculation_3("aesthetics"))
print("Average sentiment of governmental posts:", avg_sentiment_calculation_3("government"))
print("\n")

print("Average calculation method 4:")
print("Average sentiment of environmental posts:", avg_sentiment_calculation_4("environment"))
print("Average sentiment of infrastructural posts:", avg_sentiment_calculation_4("infrastructure"))
print("Average sentiment of housing-related posts:", avg_sentiment_calculation_4("housing"))
print("Average sentiment of economic posts:", avg_sentiment_calculation_4("economy"))
print("Average sentiment of life-quality-related posts:", avg_sentiment_calculation_4("life quality"))
print("Average sentiment of aesthetics-related posts:", avg_sentiment_calculation_4("aesthetics"))
print("Average sentiment of governmental posts:", avg_sentiment_calculation_4("government"))

Average calculation method 1:
Average sentiment of environmental posts: 0.0
Average sentiment of infrastructural posts: -0.2857142857142857
Average sentiment of housing-related posts: -0.25
Average sentiment of economic posts: -0.5714285714285714
Average sentiment of life-quality-related posts: -0.36363636363636365
Average sentiment of aesthetics-related posts: -0.5
Average sentiment of governmental posts: 0.0


Average calculation method 2:
Average sentiment of environmental posts: 0.0
Average sentiment of infrastructural posts: -0.14040343250547135
Average sentiment of housing-related posts: -0.18624934057394663
Average sentiment of economic posts: -0.3702814536435263
Average sentiment of life-quality-related posts: -0.24279641021381726
Average sentiment of aesthetics-related posts: -0.4823397099971771
Average sentiment of governmental posts: 0.0


Average calculation method 3:
Average sentiment of environmental posts: 0.0
Average sentiment of infrastructural posts: -0.19731166824650